In [ ]:
# 🔧 Setup: Run this cell first!
# Check GPU availability and install dependencies

import torch
import sys

# Check GPU
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"✅ GPU available: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    device = torch.device('cpu')
    print("⚠️ No GPU detected. Some cells may run slowly.")
    print("   Go to Runtime → Change runtime type → GPU")

print(f"\n📦 Python {sys.version.split()[0]}")
print(f"🔥 PyTorch {torch.__version__}")

# Set random seeds for reproducibility
import random
import numpy as np

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

print(f"🎲 Random seed set to {SEED}")

%matplotlib inline

# The ELBO, KL Divergence & Reparameterization Trick

**Notebook 2 of 4** in the Vizuara series on *Variational Autoencoders From Scratch*

In this notebook, we will derive and implement the three pillars of VAE training: the Evidence Lower Bound (ELBO), the KL divergence regularizer, and the reparameterization trick that makes it all trainable with gradient descent.

By the end, you will have a deep, hands-on understanding of every term in the VAE loss function -- and you will have built each piece from scratch.

## 1. Why Does This Matter?

Before we dive in, let us see *why* we need the ELBO at all.

In Notebook 1, we built a regular autoencoder that maps images to single points in a latent space. The problem? When we sample random points to generate new images, we get garbage -- because the latent space is full of "dead zones" that the decoder has never seen.

The VAE fixes this by making the encoder output a *distribution* instead of a point. But this creates a new question: **how do we train this probabilistic model?**

We cannot just minimize pixel error any more. We need an objective function that simultaneously:
1. Makes reconstructions accurate (decoded images should match originals)
2. Keeps the latent space well-organized (no dead zones)

The **ELBO** is that objective function. It elegantly balances both goals with exactly two terms -- and we will derive, visualize, and implement every piece of it.

Let us start with a quick preview of what the ELBO looks like in action.

In [ ]:
import torch
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

torch.manual_seed(42)
np.random.seed(42)

# --- Preview: The two forces of the ELBO ---
# Reconstruction wants the decoder to perfectly reconstruct the input.
# KL divergence wants the encoder to stay close to a standard normal.

# Simulate a training run: reconstruction loss drops while KL rises, then both settle
epochs = np.arange(1, 51)
recon_loss = 250 * np.exp(-0.08 * epochs) + 90 + np.random.normal(0, 3, len(epochs))
kl_loss = 25 * (1 - np.exp(-0.12 * epochs)) + np.random.normal(0, 1, len(epochs))
total_loss = recon_loss + kl_loss

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].plot(epochs, recon_loss, color='#2196F3', linewidth=2)
axes[0].set_title('Reconstruction Loss', fontsize=13, fontweight='bold')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_ylim(0, 350)

axes[1].plot(epochs, kl_loss, color='#FF5722', linewidth=2)
axes[1].set_title('KL Divergence', fontsize=13, fontweight='bold')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].set_ylim(0, 40)

axes[2].plot(epochs, total_loss, color='#4CAF50', linewidth=2)
axes[2].set_title('Total ELBO Loss', fontsize=13, fontweight='bold')
axes[2].set_xlabel('Epoch')
axes[2].set_ylabel('Loss')
axes[2].set_ylim(0, 350)

plt.suptitle('The Two Forces of VAE Training', fontsize=15, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print("Reconstruction loss DECREASES as the decoder improves.")
print("KL divergence INCREASES as the encoder learns to use the latent space.")
print("The ELBO balances both -- and that balance is what makes VAEs work.")

## 2. Building Intuition -- Why We Need a Training Objective

Let us think about this carefully before touching any math.

In a regular autoencoder, the training objective is simple: minimize the difference between the input image and the reconstructed image. This is just mean squared error or binary cross-entropy between pixels. Easy.

But a VAE is different. The encoder does not output a single point -- it outputs a **probability distribution** described by a mean $\mu$ and standard deviation $\sigma$. We then *sample* a latent code $z$ from this distribution and pass it to the decoder.

This creates two competing pressures:

**Pressure 1: Reconstruction quality.** The decoder wants precise, informative latent codes so it can reconstruct the image perfectly. If the encoder's distribution is very narrow (small $\sigma$), then every sample is close to $\mu$, and the decoder gets a consistent, predictable input. This is great for reconstruction.

**Pressure 2: Latent space regularity.** But if every image's distribution is narrow and far from the origin, the latent space is just a regular autoencoder in disguise -- full of dead zones. We need the distributions to overlap and cover the space smoothly. This means pushing them toward a standard normal distribution $\mathcal{N}(0, I)$.

Think of it as a GPS analogy:
- **Reconstruction loss** is like "arrive at the destination as accurately as possible."
- **KL divergence** is like "but stay on the main roads" (the standard normal prior).

If you only cared about reaching the destination, you would take bizarre shortcuts through fields and alleys. If you only cared about staying on main roads, you would never reach your specific destination. The ELBO combines both: **reach your destination while staying on well-traveled paths.**

This tension between reconstruction and regularization is the heart of VAE training. Now let us formalize it mathematically.

## 3. The Mathematics -- Deriving the ELBO Step by Step

We will derive the ELBO from first principles. Every step will be followed by a plain-English explanation and a numerical example.

### 3.1 What Are We Trying to Maximize?

We want our model to assign high probability to real data. Mathematically, we want to maximize the **log-likelihood** of the data:

$$\log p_\theta(x)$$

In plain English: "How likely is data point $x$ under our model?" We want this to be as large as possible.

But computing this directly requires an intractable integral:

$$p_\theta(x) = \int p_\theta(x|z) \, p(z) \, dz$$

This says: to find the probability of image $x$, we would need to try *every possible* latent code $z$, decode each one, and sum up the probabilities. This is impossible for continuous, high-dimensional $z$.

**Numerical intuition:** Imagine a 2D latent space. Even if we discretized it into a 100x100 grid, we would need 10,000 forward passes through the decoder. For a 20-dimensional latent space? That is $100^{20}$ evaluations -- far more than atoms in the universe.

### 3.2 Step 1 -- Introduce the Encoder

Since we cannot compute the integral directly, we introduce our encoder $q_\phi(z|x)$ as a helper. We multiply and divide by it inside the integral:

$$\log p_\theta(x) = \log \int p_\theta(x, z) \, dz = \log \int \frac{p_\theta(x, z)}{q_\phi(z|x)} \cdot q_\phi(z|x) \, dz$$

In plain English: "We have not changed anything mathematically -- we just multiplied by 1 in a clever way. But now the integral looks like an *expectation* under $q_\phi(z|x)$, which we can approximate by sampling."

In [ ]:
# Numerical illustration: multiplying and dividing by the same thing = identity
# This is the mathematical trick that gets the derivation started.

p_xz = 0.03   # p(x, z) -- joint probability for some specific z
q_zx = 0.15   # q(z|x) -- encoder probability for that same z

# Original: just p(x,z)
print(f"p(x, z) = {p_xz}")

# Multiply and divide by q(z|x):
print(f"p(x, z) / q(z|x) * q(z|x) = {p_xz / q_zx} * {q_zx} = {(p_xz / q_zx) * q_zx}")
print(f"\nSame number! But now we can treat q(z|x) as a sampling distribution.")

### 3.3 Step 2 -- Apply Jensen's Inequality

Jensen's inequality states: for a concave function like $\log$, the log of an expectation is at least as large as the expectation of the log.

$$\log \mathbb{E}[X] \geq \mathbb{E}[\log X]$$

Applying this to our integral:

$$\log p_\theta(x) \geq \mathbb{E}_{q_\phi(z|x)}\left[\log \frac{p_\theta(x, z)}{q_\phi(z|x)}\right]$$

In plain English: "The true log-likelihood is *at least as large as* the right-hand side. So if we maximize the right-hand side (the ELBO), we are pushing up the true objective as well."

In [ ]:
# Demonstrate Jensen's inequality with concrete numbers.
# For the concave function log: log(E[X]) >= E[log(X)]

values = np.array([1.0, 2.0, 5.0, 8.0])
weights = np.array([0.25, 0.25, 0.25, 0.25])  # uniform weights

# Left side: log of the expectation
expectation = np.sum(weights * values)
log_of_expectation = np.log(expectation)

# Right side: expectation of the log
log_values = np.log(values)
expectation_of_log = np.sum(weights * log_values)

print("Jensen's Inequality Demonstration")
print("=" * 40)
print(f"Values: {values}")
print(f"Weights: {weights}")
print()
print(f"E[X]      = {expectation:.4f}")
print(f"log(E[X]) = {log_of_expectation:.4f}   <-- LEFT side")
print()
print(f"log(X)    = {np.round(log_values, 4)}")
print(f"E[log(X)] = {expectation_of_log:.4f}   <-- RIGHT side")
print()
print(f"log(E[X]) >= E[log(X)]  =>  {log_of_expectation:.4f} >= {expectation_of_log:.4f}  =>  {log_of_expectation >= expectation_of_log}")
print()
print("The gap between these two is related to the KL divergence.")
print("By maximizing the lower bound (ELBO), we push up the true log-likelihood.")

### 3.4 Step 3 -- Expand the Joint Probability

We can factorize the joint probability as $p_\theta(x, z) = p_\theta(x|z) \cdot p(z)$. Substituting:

$$\log p_\theta(x) \geq \mathbb{E}_{q_\phi(z|x)}\left[\log p_\theta(x|z) + \log p(z) - \log q_\phi(z|x)\right]$$

In plain English: "We split the joint probability into two parts -- the likelihood of $x$ given $z$ (how well the decoder reconstructs), and the prior probability of $z$ (how 'normal' is this latent code)."

### 3.5 Step 4 -- Rearrange into Two Beautiful Terms

Grouping the $p(z)$ and $q_\phi(z|x)$ terms together, we arrive at:

$$\log p_\theta(x) \geq \underbrace{\mathbb{E}_{q_\phi(z|x)}[\log p_\theta(x|z)]}_{\text{Reconstruction}} - \underbrace{D_{KL}(q_\phi(z|x) \,\|\, p(z))}_{\text{Regularization}}$$

**This is the Evidence Lower Bound (ELBO).** It has exactly two terms:

**Term 1: Reconstruction Loss** -- $\mathbb{E}_{q_\phi(z|x)}[\log p_\theta(x|z)]$

"Sample a latent code $z$ from the encoder, pass it through the decoder, and measure how well the output matches the input." Higher is better. In practice, we use the negative binary cross-entropy (for binary images) or negative MSE (for continuous images).

**Term 2: KL Divergence** -- $D_{KL}(q_\phi(z|x) \,\|\, p(z))$

"How different is the encoder's output distribution from the standard normal prior?" Lower is better. This term acts as a regularizer -- it prevents the encoder from collapsing to a trivial mapping.

We *maximize* the ELBO, which means we *maximize* reconstruction (make the first term large) while *minimizing* KL divergence (make the second term small). In practice, we negate the ELBO and *minimize* it as a loss:

$$\mathcal{L}_{\text{VAE}} = -\mathbb{E}_{q_\phi(z|x)}[\log p_\theta(x|z)] + D_{KL}(q_\phi(z|x) \,\|\, p(z))$$

Let us now implement both terms with numerical examples.

## 4. Let's Build It -- Implementing Each Component

### 4.1 Reconstruction Loss (Binary Cross-Entropy)

For binary images (like MNIST), the reconstruction loss is the **binary cross-entropy** (BCE) between the true pixel values $x_i$ and the decoder's predicted probabilities $p_i$:

$$\mathcal{L}_{\text{recon}} = -\sum_{i=1}^{D} \left[ x_i \log(p_i) + (1 - x_i) \log(1 - p_i) \right]$$

In plain English: for each pixel, we ask "if the true value is 1, how confident was the decoder that it is 1? If the true value is 0, how confident was the decoder that it is 0?" We sum up these log-probabilities across all pixels.

Let us compute this by hand for a tiny 4-pixel image.

In [ ]:
import torch
import torch.nn.functional as F

torch.manual_seed(42)

# --- Reconstruction Loss: Step-by-step computation ---

# A tiny 4-pixel image and the decoder's predictions
x = torch.tensor([1.0, 0.0, 1.0, 1.0])          # true pixel values
p = torch.tensor([0.9, 0.1, 0.8, 0.95])          # decoder's predicted probabilities

print("Reconstruction Loss (Binary Cross-Entropy) -- Per Pixel")
print("=" * 60)

# Compute BCE for each pixel: -[x_i * log(p_i) + (1 - x_i) * log(1 - p_i)]
for i in range(len(x)):
    bce_i = -(x[i] * torch.log(p[i]) + (1 - x[i]) * torch.log(1 - p[i]))
    print(f"Pixel {i+1}: x={x[i]:.0f}, p={p[i]:.2f}")
    print(f"  BCE = -[{x[i]:.0f} * log({p[i]:.2f}) + {1-x[i]:.0f} * log({1-p[i]:.2f})]")
    print(f"      = -[{x[i] * torch.log(p[i]):.4f} + {(1-x[i]) * torch.log(1-p[i]):.4f}]")
    print(f"      = {bce_i:.4f}")
    print()

# Total reconstruction loss
total_bce = F.binary_cross_entropy(p, x, reduction='sum')
print(f"Total reconstruction loss = {total_bce:.4f}")
print()
print("Interpretation: a loss of 0.484 means the decoder is doing a GOOD job.")
print("Perfect predictions (p = x exactly) would give a loss of 0.")
print("Random predictions (p = 0.5 for all) would give a much higher loss.")

# Verify: what would random predictions give?
p_random = torch.tensor([0.5, 0.5, 0.5, 0.5])
random_bce = F.binary_cross_entropy(p_random, x, reduction='sum')
print(f"\nWith random predictions: total loss = {random_bce:.4f}")

### 4.2 KL Divergence -- Closed-Form Solution

The KL divergence measures how different the encoder's distribution $q_\phi(z|x) = \mathcal{N}(\mu, \sigma^2)$ is from the standard normal prior $p(z) = \mathcal{N}(0, 1)$.

The beautiful thing is that for two Gaussians, KL divergence has a **closed-form** solution -- no sampling needed:

$$D_{KL} = -\frac{1}{2}\left(1 + \log(\sigma^2) - \mu^2 - \sigma^2\right)$$

In plain English: "This formula measures the 'distance' between the encoder's Gaussian and the standard normal. It is zero only when $\mu = 0$ and $\sigma = 1$ -- meaning the encoder perfectly matches the prior."

For a latent space of dimension $J$, we sum over all dimensions:

$$D_{KL} = -\frac{1}{2} \sum_{j=1}^{J}\left(1 + \log(\sigma_j^2) - \mu_j^2 - \sigma_j^2\right)$$

Let us compute two concrete examples.

In [ ]:
import torch

torch.manual_seed(42)

# --- KL Divergence: Closed-form computation ---

def kl_divergence_manual(mu, sigma):
    """Compute KL divergence between N(mu, sigma^2) and N(0, 1) for a single dimension."""
    return -0.5 * (1 + torch.log(sigma**2) - mu**2 - sigma**2)


print("KL Divergence -- Closed Form Examples")
print("=" * 55)

# Example 1: mu = 0.5, sigma = 1.2  (slightly off from standard normal)
mu1, sigma1 = torch.tensor(0.5), torch.tensor(1.2)
kl1 = kl_divergence_manual(mu1, sigma1)
print(f"\nExample 1: mu = {mu1:.1f}, sigma = {sigma1:.1f}")
print(f"  D_KL = -0.5 * (1 + log({sigma1**2:.2f}) - {mu1**2:.2f} - {sigma1**2:.2f})")
print(f"       = -0.5 * (1 + {torch.log(sigma1**2):.4f} - {mu1**2:.4f} - {sigma1**2:.4f})")
inner = 1 + torch.log(sigma1**2) - mu1**2 - sigma1**2
print(f"       = -0.5 * ({inner:.4f})")
print(f"       = {kl1:.4f}")
print(f"  This means the encoder's distribution is SLIGHTLY off from the standard normal.")

# Example 2: mu = 0, sigma = 1  (perfect standard normal)
mu2, sigma2 = torch.tensor(0.0), torch.tensor(1.0)
kl2 = kl_divergence_manual(mu2, sigma2)
print(f"\nExample 2: mu = {mu2:.1f}, sigma = {sigma2:.1f}")
print(f"  D_KL = -0.5 * (1 + log({sigma2**2:.2f}) - {mu2**2:.2f} - {sigma2**2:.2f})")
print(f"       = -0.5 * (1 + {torch.log(sigma2**2):.4f} - {mu2**2:.4f} - {sigma2**2:.4f})")
inner2 = 1 + torch.log(sigma2**2) - mu2**2 - sigma2**2
print(f"       = -0.5 * ({inner2:.4f})")
print(f"       = {kl2:.4f}")
print(f"  Perfect match! KL = 0 when the encoder IS the standard normal.")

# Example 3: mu = 3.0, sigma = 0.1  (far away, very narrow -- bad!)
mu3, sigma3 = torch.tensor(3.0), torch.tensor(0.1)
kl3 = kl_divergence_manual(mu3, sigma3)
print(f"\nExample 3: mu = {mu3:.1f}, sigma = {sigma3:.1f}")
print(f"  D_KL = {kl3:.4f}")
print(f"  Very high! The encoder is far from standard normal -- strong penalty.")

Now let us visualize how the KL divergence changes as we vary $\mu$ and $\sigma$. This gives us an intuitive map of the "penalty landscape."

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

torch.manual_seed(42)

# --- Visualization 1: KL Divergence as a function of mu and sigma ---

mu_range = np.linspace(-3, 3, 100)
sigma_range = np.linspace(0.1, 3.0, 100)
MU, SIGMA = np.meshgrid(mu_range, sigma_range)

# KL divergence: -0.5 * (1 + log(sigma^2) - mu^2 - sigma^2)
KL = -0.5 * (1 + np.log(SIGMA**2) - MU**2 - SIGMA**2)

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Heatmap
im = axes[0].pcolormesh(MU, SIGMA, KL, cmap='YlOrRd', shading='auto', vmin=0, vmax=5)
axes[0].set_xlabel('$\\mu$ (mean)', fontsize=12)
axes[0].set_ylabel('$\\sigma$ (std dev)', fontsize=12)
axes[0].set_title('KL Divergence Heatmap', fontsize=14, fontweight='bold')
axes[0].plot(0, 1, 'k*', markersize=15, label='$\\mu$=0, $\\sigma$=1 (KL=0)')
axes[0].legend(fontsize=11)
plt.colorbar(im, ax=axes[0], label='$D_{KL}$')

# Cross-sections
sigmas_to_plot = [0.5, 1.0, 1.5, 2.0]
colors = ['#E91E63', '#2196F3', '#4CAF50', '#FF9800']
for sigma_val, color in zip(sigmas_to_plot, colors):
    kl_vals = -0.5 * (1 + np.log(sigma_val**2) - mu_range**2 - sigma_val**2)
    axes[1].plot(mu_range, kl_vals, color=color, linewidth=2, label=f'$\\sigma$={sigma_val}')

axes[1].set_xlabel('$\\mu$ (mean)', fontsize=12)
axes[1].set_ylabel('$D_{KL}$', fontsize=12)
axes[1].set_title('KL Divergence vs $\\mu$ (for fixed $\\sigma$)', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=11)
axes[1].set_ylim(0, 5)
axes[1].axhline(y=0, color='gray', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()

print("Key observations:")
print("1. KL = 0 only at mu=0, sigma=1 (the star) -- perfect match to prior")
print("2. Moving mu away from 0 increases KL (mean penalty)")
print("3. Making sigma too small or too large increases KL (spread penalty)")
print("4. The minimum for each sigma curve is always at mu=0")

### 4.3 The Reparameterization Trick

We have the ELBO and we know how to compute both terms. But there is one more technical hurdle before we can train.

**The Problem:** During the forward pass, we need to *sample* $z$ from the encoder's distribution $\mathcal{N}(\mu, \sigma^2)$. But sampling is a random operation, and **random operations are not differentiable**. If gradients cannot flow through the sampling step, backpropagation breaks down and we cannot train the encoder.

**The Solution:** Instead of sampling $z$ directly, we reparameterize it as a *deterministic* function of $\mu$, $\sigma$, and a random noise variable $\epsilon$:

$$z = \mu + \sigma \odot \epsilon, \quad \text{where } \epsilon \sim \mathcal{N}(0, I)$$

In plain English: "We move the randomness into $\epsilon$, which does not depend on any learnable parameters. The quantities $\mu$ and $\sigma$ are deterministic outputs of the encoder, so gradients flow through them freely."

**Numerical example:**

Suppose $\mu = [2.1, -0.5]$, $\sigma = [0.3, 0.2]$, and we sample $\epsilon = [0.7, -1.1]$.

$$z = [2.1 + 0.3 \times 0.7, \; -0.5 + 0.2 \times (-1.1)]$$
$$z = [2.1 + 0.21, \; -0.5 + (-0.22)]$$
$$z = [2.31, -0.72]$$

The gradient $\frac{\partial z}{\partial \mu} = 1$ and $\frac{\partial z}{\partial \sigma} = \epsilon$, so gradients are well-defined.

In [ ]:
import torch

torch.manual_seed(42)

# --- Reparameterization Trick: Step by step ---

mu = torch.tensor([2.1, -0.5], requires_grad=True)
sigma = torch.tensor([0.3, 0.2], requires_grad=True)

# The "wrong" way: sample directly (cannot backprop)
print("=" * 60)
print("THE REPARAMETERIZATION TRICK")
print("=" * 60)

# The "right" way: reparameterize
epsilon = torch.tensor([0.7, -1.1])  # fixed for demonstration
z = mu + sigma * epsilon

print(f"\nInputs:")
print(f"  mu    = {mu.data.tolist()}")
print(f"  sigma = {sigma.data.tolist()}")
print(f"  epsilon (sampled from N(0,I)) = {epsilon.tolist()}")
print(f"\nComputation: z = mu + sigma * epsilon")
print(f"  z[0] = {mu[0].item():.1f} + {sigma[0].item():.1f} * {epsilon[0].item():.1f} = {z[0].item():.2f}")
print(f"  z[1] = {mu[1].item():.1f} + {sigma[1].item():.1f} * {epsilon[1].item():.1f} = {z[1].item():.2f}")
print(f"  z    = [{z[0].item():.2f}, {z[1].item():.2f}]")

# Now show that gradients flow through mu and sigma
loss = z.sum()  # dummy loss to trigger backward
loss.backward()

print(f"\nGradient check (this is the key!):")
print(f"  dz/d(mu)    = {mu.grad.tolist()}  (always 1 -- gradient flows!)")
print(f"  dz/d(sigma) = {sigma.grad.tolist()}  (= epsilon -- gradient flows!)")
print(f"  dz/d(epsilon) = not tracked (epsilon has no grad)")
print(f"\nThe randomness is in epsilon, which is NOT a learnable parameter.")
print(f"Gradients flow through mu and sigma, so the encoder can be trained.")

Let us now visualize the reparameterization trick more concretely by showing that gradients *do* flow through $\mu$ and $\sigma$ but *do not* flow through $\epsilon$.

In [ ]:
import torch
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

torch.manual_seed(42)

# --- Visualization 2: Gradient flow in reparameterization ---

# Run the reparameterization trick multiple times and collect gradients
n_samples = 200
mu = torch.tensor([1.5], requires_grad=True)
sigma = torch.tensor([0.8], requires_grad=True)

z_values = []
grad_mu_values = []
grad_sigma_values = []

for i in range(n_samples):
    # Fresh computation graph each time
    mu_val = mu.detach().requires_grad_(True)
    sigma_val = sigma.detach().requires_grad_(True)

    eps = torch.randn(1)
    z = mu_val + sigma_val * eps

    # Compute gradient of z w.r.t. mu and sigma
    z.backward()

    z_values.append(z.item())
    grad_mu_values.append(mu_val.grad.item())
    grad_sigma_values.append(sigma_val.grad.item())

z_values = np.array(z_values)
grad_mu_values = np.array(grad_mu_values)
grad_sigma_values = np.array(grad_sigma_values)

fig, axes = plt.subplots(1, 3, figsize=(16, 4.5))

# Panel 1: Distribution of z samples
axes[0].hist(z_values, bins=30, color='#7E57C2', alpha=0.7, edgecolor='white')
axes[0].axvline(x=1.5, color='red', linewidth=2, linestyle='--', label=f'$\\mu$ = 1.5')
axes[0].set_xlabel('z value', fontsize=12)
axes[0].set_ylabel('Count', fontsize=12)
axes[0].set_title('Samples from $z = \\mu + \\sigma\\epsilon$', fontsize=13, fontweight='bold')
axes[0].legend(fontsize=11)

# Panel 2: dz/d(mu) is always 1
axes[1].scatter(range(n_samples), grad_mu_values, s=5, color='#2196F3', alpha=0.7)
axes[1].axhline(y=1.0, color='red', linewidth=2, linestyle='--', label='Expected: 1.0')
axes[1].set_xlabel('Sample index', fontsize=12)
axes[1].set_ylabel('$\\partial z / \\partial \\mu$', fontsize=12)
axes[1].set_title('Gradient w.r.t. $\\mu$ (always 1)', fontsize=13, fontweight='bold')
axes[1].set_ylim(0.5, 1.5)
axes[1].legend(fontsize=11)

# Panel 3: dz/d(sigma) = epsilon (varies)
axes[2].scatter(range(n_samples), grad_sigma_values, s=5, color='#FF5722', alpha=0.7)
axes[2].axhline(y=0.0, color='gray', linewidth=1, linestyle='--')
axes[2].set_xlabel('Sample index', fontsize=12)
axes[2].set_ylabel('$\\partial z / \\partial \\sigma$', fontsize=12)
axes[2].set_title('Gradient w.r.t. $\\sigma$ (= $\\epsilon$, varies)', fontsize=13, fontweight='bold')
axes[2].legend(['$\\epsilon \\sim \\mathcal{N}(0,1)$'], fontsize=11)

plt.suptitle('Reparameterization Trick: Gradients Flow Through $\\mu$ and $\\sigma$',
             fontsize=14, fontweight='bold', y=1.03)
plt.tight_layout()
plt.show()

print("Key takeaway:")
print("  dz/d(mu) = 1.0 always -- clean, constant gradient signal")
print("  dz/d(sigma) = epsilon -- noisy but well-defined gradient signal")
print("  Both are differentiable, so backpropagation works!")

### 4.4 The ELBO Balance -- Reconstruction vs KL Tradeoff

One of the most important insights about the ELBO is the *tension* between its two terms. Let us visualize this tradeoff directly.

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

torch.manual_seed(42)

# --- Visualization 3: ELBO balance as sigma varies ---
# For a fixed mu, show how reconstruction and KL trade off as sigma changes

mu_fixed = torch.tensor([1.0])
sigma_range = torch.linspace(0.01, 3.0, 200)

# Simulated reconstruction loss: as sigma grows, z samples are more spread out,
# so the decoder gets noisier inputs and reconstruction worsens.
# We model this as: recon_loss ~ base + scale * sigma^2
recon_base = 80.0
recon_scale = 30.0
recon_losses = recon_base + recon_scale * sigma_range**2

# KL divergence: closed form
kl_losses = -0.5 * (1 + torch.log(sigma_range**2) - mu_fixed**2 - sigma_range**2)

# Total ELBO loss (negated ELBO = recon_loss + KL)
total_losses = recon_losses + kl_losses

fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(sigma_range.numpy(), recon_losses.numpy(), color='#2196F3', linewidth=2.5, label='Reconstruction Loss')
ax.plot(sigma_range.numpy(), kl_losses.numpy(), color='#FF5722', linewidth=2.5, label='KL Divergence')
ax.plot(sigma_range.numpy(), total_losses.numpy(), color='#4CAF50', linewidth=2.5, linestyle='--', label='Total Loss (Recon + KL)')

# Mark the optimal sigma (minimum total loss)
min_idx = torch.argmin(total_losses)
opt_sigma = sigma_range[min_idx].item()
opt_loss = total_losses[min_idx].item()
ax.plot(opt_sigma, opt_loss, 'k*', markersize=15, zorder=5)
ax.annotate(f'Optimal $\\sigma$ = {opt_sigma:.2f}', xy=(opt_sigma, opt_loss),
            xytext=(opt_sigma + 0.5, opt_loss + 20), fontsize=12,
            arrowprops=dict(arrowstyle='->', color='black'))

ax.set_xlabel('$\\sigma$ (encoder standard deviation)', fontsize=13)
ax.set_ylabel('Loss', fontsize=13)
ax.set_title(f'The ELBO Balancing Act (fixed $\\mu$ = {mu_fixed.item():.1f})', fontsize=14, fontweight='bold')
ax.legend(fontsize=12)
ax.set_ylim(0, 200)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"At the optimal sigma = {opt_sigma:.2f}:")
print(f"  Reconstruction loss = {recon_losses[min_idx].item():.2f}")
print(f"  KL divergence       = {kl_losses[min_idx].item():.2f}")
print(f"  Total ELBO loss     = {opt_loss:.2f}")
print()
print("If sigma is too small: KL is high (far from N(0,1)), but reconstruction is good.")
print("If sigma is too large: KL might decrease, but reconstruction suffers.")
print("The ELBO finds the sweet spot that balances both pressures.")

## 5. Your Turn -- TODO Exercises

Now it is your turn to implement these components from scratch. We provide scaffolding and verification cells to check your work.

### TODO 1: Implement KL Divergence from Scratch

Given the closed-form formula, compute the KL divergence for a batch of encoder outputs. The formula for a single latent dimension is:

$$D_{KL} = -\frac{1}{2}\left(1 + \log(\sigma^2) - \mu^2 - \sigma^2\right)$$

For a latent space of dimension $J$ and a batch of $N$ samples, sum over all dimensions and all samples.

**Hint:** In practice, neural networks output $\log(\sigma^2)$ (log-variance) instead of $\sigma$ directly. So the formula becomes:

$$D_{KL} = -\frac{1}{2}\left(1 + \log\sigma^2 - \mu^2 - e^{\log\sigma^2}\right)$$

In [ ]:
import torch

torch.manual_seed(42)

def kl_divergence_todo(mu, logvar):
    """
    Compute the KL divergence between q(z|x) = N(mu, exp(logvar)) and p(z) = N(0, I).

    Args:
        mu: Tensor of shape (batch_size, latent_dim) -- encoder means
        logvar: Tensor of shape (batch_size, latent_dim) -- encoder log-variances

    Returns:
        Scalar tensor: total KL divergence summed over batch and dimensions
    """
    # ============================================================
    # TODO: Implement the KL divergence formula here.
    #
    # The closed-form KL for one dimension is:
    #   -0.5 * (1 + logvar - mu^2 - exp(logvar))
    #
    # Sum over all latent dimensions AND all batch samples.
    #
    # Replace the line below with your implementation:
    # ============================================================
    kl = None  # YOUR CODE HERE

    return kl


# --- Test data ---
# Batch of 4 samples, each with 3 latent dimensions
mu_test = torch.tensor([
    [0.0, 0.0, 0.0],     # perfect standard normal
    [0.5, -0.3, 0.1],    # slightly off
    [2.0, 1.5, -1.0],    # far from standard normal
    [0.0, 0.0, 0.0],     # perfect again, but with sigma != 1
])
logvar_test = torch.tensor([
    [0.0, 0.0, 0.0],           # sigma = 1 (log(1) = 0)
    [0.3646, 0.1823, -0.2231], # sigma ~ [1.2, 1.1, 0.9]
    [0.0, 0.0, 0.0],           # sigma = 1 but mu is far
    [-2.0, -1.0, 1.0],         # sigma = [0.37, 0.61, 1.65]
])

kl_result = kl_divergence_todo(mu_test, logvar_test)
print(f"Your KL divergence result: {kl_result}")

#### Verification Cell for TODO 1

Run this cell to check your implementation. If you see all green, you got it right!

In [ ]:
# --- Verification for TODO 1 ---

# Reference implementation
def kl_divergence_reference(mu, logvar):
    return -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

expected = kl_divergence_reference(mu_test, logvar_test)

print("Verification for TODO 1: KL Divergence")
print("=" * 50)

if kl_result is None:
    print("You have not implemented kl_divergence_todo yet!")
    print(f"Expected result: {expected.item():.4f}")
else:
    try:
        assert torch.isclose(kl_result, expected, atol=1e-3), \
            f"Mismatch! Got {kl_result.item():.4f}, expected {expected.item():.4f}"
        print(f"Your result:     {kl_result.item():.4f}")
        print(f"Expected result: {expected.item():.4f}")
        print("PASSED! Your KL divergence implementation is correct.")
    except AssertionError as e:
        print(f"FAILED: {e}")

# Break down by sample to help debug
print("\nBreakdown by sample:")
for i in range(mu_test.shape[0]):
    kl_i = -0.5 * torch.sum(1 + logvar_test[i] - mu_test[i].pow(2) - logvar_test[i].exp())
    print(f"  Sample {i+1}: mu={mu_test[i].tolist()}, logvar={logvar_test[i].tolist()}")
    print(f"    KL = {kl_i.item():.4f}")

### TODO 2: Implement the Reparameterization Trick and Verify Gradients

Implement the reparameterization trick: given $\mu$ and $\log(\sigma^2)$, sample $z$ such that gradients flow back to $\mu$ and $\sigma$.

Remember: $z = \mu + \sigma \odot \epsilon$, where $\epsilon \sim \mathcal{N}(0, I)$ and $\sigma = \exp(0.5 \cdot \log\sigma^2)$.

In [ ]:
import torch

torch.manual_seed(42)

def reparameterize_todo(mu, logvar):
    """
    Sample z from q(z|x) = N(mu, exp(logvar)) using the reparameterization trick.

    Args:
        mu: Tensor of shape (batch_size, latent_dim) -- encoder means
        logvar: Tensor of shape (batch_size, latent_dim) -- encoder log-variances

    Returns:
        z: Tensor of shape (batch_size, latent_dim) -- sampled latent codes
    """
    # ============================================================
    # TODO: Implement the reparameterization trick.
    #
    # Step 1: Compute std = exp(0.5 * logvar)
    # Step 2: Sample epsilon from N(0, I) with the same shape as std
    #         Use torch.randn_like(std) for this.
    # Step 3: Compute z = mu + std * epsilon
    #
    # Replace the line below with your implementation:
    # ============================================================
    z = None  # YOUR CODE HERE

    return z


# --- Test ---
mu_test2 = torch.tensor([[2.1, -0.5]], requires_grad=True)
logvar_test2 = torch.tensor([[2 * torch.log(torch.tensor(0.3)).item(),
                               2 * torch.log(torch.tensor(0.2)).item()]])

z_sample = reparameterize_todo(mu_test2, logvar_test2)
print(f"Sampled z: {z_sample}")

#### Verification Cell for TODO 2

Run this cell to verify that gradients flow correctly through your reparameterization.

In [ ]:
# --- Verification for TODO 2 ---

print("Verification for TODO 2: Reparameterization Trick")
print("=" * 55)

if z_sample is None:
    print("You have not implemented reparameterize_todo yet!")
else:
    # Check 1: Output shape is correct
    assert z_sample.shape == mu_test2.shape, \
        f"Shape mismatch: got {z_sample.shape}, expected {mu_test2.shape}"
    print("Shape check: PASSED")

    # Check 2: Gradients flow through mu
    try:
        dummy_loss = z_sample.sum()
        dummy_loss.backward()
        assert mu_test2.grad is not None, "No gradient for mu!"
        assert torch.all(mu_test2.grad == 1.0), \
            f"Expected dz/dmu = 1, got {mu_test2.grad}"
        print(f"Gradient check (dz/dmu = {mu_test2.grad.tolist()}): PASSED")
    except Exception as e:
        print(f"Gradient check FAILED: {e}")

    # Check 3: z is different from mu (sampling adds noise)
    is_different = not torch.allclose(z_sample.detach(), mu_test2.detach())
    print(f"Stochasticity check (z != mu): {'PASSED' if is_different else 'WARNING: z equals mu exactly'}")

    # Check 4: Multiple samples produce different results
    torch.manual_seed(0)
    z1 = mu_test2.detach() + torch.exp(0.5 * logvar_test2) * torch.randn_like(mu_test2)
    torch.manual_seed(1)
    z2 = mu_test2.detach() + torch.exp(0.5 * logvar_test2) * torch.randn_like(mu_test2)
    print(f"Different seeds give different z: {'PASSED' if not torch.allclose(z1, z2) else 'FAILED'}")

    print("\nAll checks passed! Your reparameterization trick works correctly.")

## 6. Putting It All Together -- The Full VAE Loss Function

Now let us combine all three components into a single, clean VAE loss function. This is the complete training objective.

In [ ]:
import torch
import torch.nn.functional as F

torch.manual_seed(42)

def vae_loss(x_recon, x, mu, logvar):
    """
    Complete VAE loss function = Reconstruction Loss + KL Divergence.

    Args:
        x_recon: Decoder output probabilities, shape (batch_size, D)
        x: Original input, shape (batch_size, D)
        mu: Encoder means, shape (batch_size, latent_dim)
        logvar: Encoder log-variances, shape (batch_size, latent_dim)

    Returns:
        total_loss: Scalar tensor
        recon_loss: Reconstruction component (for logging)
        kl_loss: KL divergence component (for logging)
    """
    # Term 1: Reconstruction loss (binary cross-entropy, summed over pixels)
    recon_loss = F.binary_cross_entropy(x_recon, x, reduction='sum')

    # Term 2: KL divergence (closed-form for Gaussian)
    kl_loss = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    # Total ELBO loss (negated ELBO -- we minimize this)
    total_loss = recon_loss + kl_loss

    return total_loss, recon_loss, kl_loss


# --- Demo: compute the loss for a synthetic mini-batch ---
batch_size = 4
D = 16  # 4x4 pixel images
latent_dim = 2

# Simulate encoder outputs
mu = torch.randn(batch_size, latent_dim)
logvar = torch.randn(batch_size, latent_dim)

# Simulate original images (binary)
x = torch.bernoulli(torch.ones(batch_size, D) * 0.5)

# Simulate decoder predictions (probabilities between 0 and 1)
x_recon = torch.sigmoid(torch.randn(batch_size, D))

# Compute loss
total, recon, kl = vae_loss(x_recon, x, mu, logvar)

print("Full VAE Loss Computation")
print("=" * 50)
print(f"Batch size: {batch_size}")
print(f"Image dimension: {D} pixels")
print(f"Latent dimension: {latent_dim}")
print()
print(f"Reconstruction loss: {recon.item():.4f}")
print(f"KL divergence:       {kl.item():.4f}")
print(f"Total ELBO loss:     {total.item():.4f}")
print()
print(f"Reconstruction share: {100 * recon.item() / total.item():.1f}%")
print(f"KL share:             {100 * kl.item() / total.item():.1f}%")
print()
print("In early training, reconstruction dominates (the decoder is learning).")
print("As training progresses, KL grows (the encoder learns structure).")

## 7. Interactive Exploration -- The Beta-VAE

In a standard VAE, both loss terms are weighted equally. But what if we could control the balance? The **beta-VAE** introduces a weighting factor $\beta$:

$$\mathcal{L}_{\beta\text{-VAE}} = \mathcal{L}_{\text{recon}} + \beta \cdot D_{KL}$$

- $\beta = 1$: Standard VAE
- $\beta < 1$: Emphasize reconstruction (sharper images, less organized latent space)
- $\beta > 1$: Emphasize regularization (blurrier images, more disentangled latent space)

Let us explore how different values of $\beta$ change the loss landscape.

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

torch.manual_seed(42)

# --- Visualization 4: Beta-VAE loss landscape ---

# We will simulate how the total loss changes as a function of mu
# for different beta values, with sigma fixed at 1.0

mu_range = np.linspace(-4, 4, 300)
sigma_fixed = 1.0

# Base reconstruction loss (quadratic in mu for simplicity -- moving away from data)
recon_loss_base = 50.0  # constant base
recon_loss_mu = 5.0 * mu_range**2  # worse reconstruction as mu moves away

# KL divergence for fixed sigma=1.0: -0.5 * (1 + 0 - mu^2 - 1) = 0.5 * mu^2
kl_base = 0.5 * mu_range**2

betas = [0.1, 0.5, 1.0, 2.0, 5.0]
colors = ['#E91E63', '#FF9800', '#4CAF50', '#2196F3', '#7E57C2']

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Panel 1: Total loss for different betas
for beta, color in zip(betas, colors):
    total = recon_loss_base + recon_loss_mu + beta * kl_base
    axes[0].plot(mu_range, total, color=color, linewidth=2, label=f'$\\beta$ = {beta}')

axes[0].set_xlabel('$\\mu$ (encoder mean)', fontsize=12)
axes[0].set_ylabel('Total Loss', fontsize=12)
axes[0].set_title('Total Loss for Different $\\beta$ Values', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=11)
axes[0].set_ylim(0, 250)
axes[0].grid(True, alpha=0.3)

# Panel 2: Component breakdown for beta = 0.1, 1.0, 5.0
beta_showcase = [0.1, 1.0, 5.0]
linestyles = ['--', '-', ':']
for beta, ls in zip(beta_showcase, linestyles):
    recon = recon_loss_base + recon_loss_mu
    kl = beta * kl_base
    axes[1].plot(mu_range, recon, color='#2196F3', linewidth=1.5, linestyle=ls, alpha=0.6)
    axes[1].plot(mu_range, kl, color='#FF5722', linewidth=1.5, linestyle=ls, alpha=0.6)

# Add labels
axes[1].plot([], [], color='#2196F3', linewidth=2, label='Reconstruction (same for all $\\beta$)')
axes[1].plot([], [], color='#FF5722', linewidth=2, linestyle='--', label='$\\beta$ KL ($\\beta$=0.1)')
axes[1].plot([], [], color='#FF5722', linewidth=2, linestyle='-', label='$\\beta$ KL ($\\beta$=1.0)')
axes[1].plot([], [], color='#FF5722', linewidth=2, linestyle=':', label='$\\beta$ KL ($\\beta$=5.0)')
axes[1].set_xlabel('$\\mu$ (encoder mean)', fontsize=12)
axes[1].set_ylabel('Loss Component', fontsize=12)
axes[1].set_title('Reconstruction vs $\\beta$-scaled KL', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=10)
axes[1].set_ylim(0, 150)
axes[1].grid(True, alpha=0.3)

plt.suptitle('Beta-VAE: Controlling the Reconstruction-Regularization Tradeoff',
             fontsize=15, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print("What beta controls:")
print("  beta < 1: The model prioritizes reconstruction. Sharper images, but")
print("            the latent space may have dead zones.")
print("  beta = 1: Standard VAE. Equal weight to both terms.")
print("  beta > 1: The model prioritizes a well-organized latent space.")
print("            Images may be blurrier, but latent factors are more disentangled.")

Let us make this even more interactive. We will compute the actual loss components for a range of $\beta$ values and show how the balance shifts.

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

torch.manual_seed(42)

# --- Interactive beta sweep ---

# Simulate a "trained" encoder output for a batch of images
batch_size = 100
latent_dim = 2

# Encoder outputs: means clustered around different digit-like regions
mu_batch = torch.randn(batch_size, latent_dim) * 1.5
logvar_batch = torch.randn(batch_size, latent_dim) * 0.3 - 0.5  # sigma around 0.7-0.9

# Simulated reconstruction quality
x_batch = torch.bernoulli(torch.ones(batch_size, 784) * 0.3)
x_recon_batch = torch.sigmoid(torch.randn(batch_size, 784) * 0.5 +
                               (x_batch - 0.5) * 3)  # correlated with true x

# Compute per-sample losses
recon_per_sample = torch.stack([
    F.binary_cross_entropy(x_recon_batch[i], x_batch[i], reduction='sum')
    for i in range(batch_size)
])
kl_per_sample = torch.stack([
    -0.5 * torch.sum(1 + logvar_batch[i] - mu_batch[i].pow(2) - logvar_batch[i].exp())
    for i in range(batch_size)
])

# Sweep beta
betas = np.logspace(-1, 1, 50)  # 0.1 to 10
mean_recon = recon_per_sample.mean().item()
mean_kl = kl_per_sample.mean().item()
total_losses = [mean_recon + b * mean_kl for b in betas]
recon_fractions = [mean_recon / (mean_recon + b * mean_kl) * 100 for b in betas]

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Panel 1: Total loss vs beta
axes[0].semilogx(betas, total_losses, color='#4CAF50', linewidth=2.5)
axes[0].axvline(x=1.0, color='gray', linestyle='--', alpha=0.5, label='$\\beta$=1 (standard VAE)')
axes[0].set_xlabel('$\\beta$', fontsize=13)
axes[0].set_ylabel('Mean Total Loss', fontsize=13)
axes[0].set_title('Total Loss vs $\\beta$', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3)

# Panel 2: Recon fraction vs beta
axes[1].semilogx(betas, recon_fractions, color='#2196F3', linewidth=2.5, label='Recon %')
axes[1].semilogx(betas, [100 - r for r in recon_fractions], color='#FF5722', linewidth=2.5, label='KL %')
axes[1].axvline(x=1.0, color='gray', linestyle='--', alpha=0.5)
axes[1].axhline(y=50, color='gray', linestyle=':', alpha=0.3)
axes[1].set_xlabel('$\\beta$', fontsize=13)
axes[1].set_ylabel('Percentage of Total Loss', fontsize=13)
axes[1].set_title('Loss Composition vs $\\beta$', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=12)
axes[1].set_ylim(0, 100)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Mean reconstruction loss: {mean_recon:.2f}")
print(f"Mean KL divergence:       {mean_kl:.2f}")
print(f"\nAt beta=0.1:  Recon is {recon_fractions[0]:.0f}% of total loss")
print(f"At beta=1.0:  Recon is ~{mean_recon/(mean_recon + mean_kl)*100:.0f}% of total loss")
print(f"At beta=10.0: Recon is {recon_fractions[-1]:.0f}% of total loss")

## 8. Final Output -- Visualization Dashboard

Let us bring everything together in a single visualization dashboard that shows all three components of the VAE loss and how they interact.

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.colors import LinearSegmentedColormap

torch.manual_seed(42)
np.random.seed(42)

fig = plt.figure(figsize=(18, 14))
gs = gridspec.GridSpec(3, 3, hspace=0.4, wspace=0.35)

# ================================================================
# Panel 1 (top-left): Reconstruction Loss Surface
# ================================================================
ax1 = fig.add_subplot(gs[0, 0])

# BCE loss as a function of predicted probability for x=1 and x=0
p_range = np.linspace(0.01, 0.99, 200)
bce_x1 = -np.log(p_range)       # when true label = 1
bce_x0 = -np.log(1 - p_range)   # when true label = 0

ax1.plot(p_range, bce_x1, color='#2196F3', linewidth=2.5, label='True pixel = 1')
ax1.plot(p_range, bce_x0, color='#FF5722', linewidth=2.5, label='True pixel = 0')
ax1.set_xlabel('Predicted probability $p$', fontsize=11)
ax1.set_ylabel('BCE Loss', fontsize=11)
ax1.set_title('Reconstruction Loss\n(Binary Cross-Entropy)', fontsize=12, fontweight='bold')
ax1.legend(fontsize=10)
ax1.set_ylim(0, 5)
ax1.grid(True, alpha=0.3)

# Annotate key points
ax1.annotate('Perfect\nprediction', xy=(0.95, -np.log(0.95)), xytext=(0.7, 1.5),
             fontsize=9, arrowprops=dict(arrowstyle='->', color='#2196F3'),
             color='#2196F3')

# ================================================================
# Panel 2 (top-center): KL Divergence Heatmap
# ================================================================
ax2 = fig.add_subplot(gs[0, 1])

mu_grid = np.linspace(-3, 3, 100)
sigma_grid = np.linspace(0.1, 3.0, 100)
MU, SIGMA = np.meshgrid(mu_grid, sigma_grid)
KL = -0.5 * (1 + np.log(SIGMA**2) - MU**2 - SIGMA**2)

im = ax2.pcolormesh(MU, SIGMA, KL, cmap='magma_r', shading='auto', vmin=0, vmax=4)
ax2.plot(0, 1, 'w*', markersize=12)
ax2.set_xlabel('$\\mu$', fontsize=11)
ax2.set_ylabel('$\\sigma$', fontsize=11)
ax2.set_title('KL Divergence\n$D_{KL}(\\mathcal{N}(\\mu,\\sigma^2) \\| \\mathcal{N}(0,1))$',
              fontsize=12, fontweight='bold')
plt.colorbar(im, ax=ax2, label='$D_{KL}$', shrink=0.8)

# ================================================================
# Panel 3 (top-right): Reparameterization Gradient Flow
# ================================================================
ax3 = fig.add_subplot(gs[0, 2])

# Show multiple reparameterized samples from same distribution
mu_val = 1.5
sigma_val = 0.6
n_eps = 500
epsilons = np.random.randn(n_eps)
z_samples = mu_val + sigma_val * epsilons

ax3.hist(z_samples, bins=40, density=True, alpha=0.6, color='#7E57C2', edgecolor='white',
         label=f'$z = \\mu + \\sigma\\epsilon$')

# Overlay the theoretical Gaussian
z_theory = np.linspace(mu_val - 3*sigma_val, mu_val + 3*sigma_val, 200)
pdf = (1 / (sigma_val * np.sqrt(2*np.pi))) * np.exp(-0.5 * ((z_theory - mu_val)/sigma_val)**2)
ax3.plot(z_theory, pdf, color='#FF5722', linewidth=2.5, label=f'$\\mathcal{{N}}({mu_val}, {sigma_val}^2)$')
ax3.axvline(mu_val, color='red', linestyle='--', linewidth=1.5, label=f'$\\mu = {mu_val}$')

# Show gradient arrows
for offset in [-1.0, -0.3, 0.5, 1.2]:
    z_pt = mu_val + sigma_val * offset
    ax3.annotate('', xy=(z_pt, 0.05), xytext=(mu_val, 0.05),
                arrowprops=dict(arrowstyle='->', color='#4CAF50', lw=1.5))

ax3.set_xlabel('$z$', fontsize=11)
ax3.set_ylabel('Density', fontsize=11)
ax3.set_title('Reparameterization\nGradient Flow', fontsize=12, fontweight='bold')
ax3.legend(fontsize=9, loc='upper left')

# ================================================================
# Panel 4 (middle, spanning full width): ELBO Training Dynamics
# ================================================================
ax4 = fig.add_subplot(gs[1, :])

epochs = np.arange(1, 101)
np.random.seed(42)

# Simulate realistic training curves
recon_curve = 550 * np.exp(-0.04 * epochs) + 85 + np.random.normal(0, 5, len(epochs))
kl_curve = 22 * (1 - np.exp(-0.06 * epochs)) + 3 + np.random.normal(0, 1, len(epochs))
kl_curve = np.maximum(kl_curve, 0)
total_curve = recon_curve + kl_curve

ax4.plot(epochs, recon_curve, color='#2196F3', linewidth=2, alpha=0.8, label='Reconstruction Loss')
ax4.plot(epochs, kl_curve, color='#FF5722', linewidth=2, alpha=0.8, label='KL Divergence')
ax4.plot(epochs, total_curve, color='#4CAF50', linewidth=2.5, label='Total ELBO Loss')

# Add phases
ax4.axvspan(1, 15, alpha=0.08, color='blue')
ax4.axvspan(15, 50, alpha=0.08, color='orange')
ax4.axvspan(50, 100, alpha=0.08, color='green')
ax4.text(7, max(total_curve) * 0.9, 'Phase 1:\nDecoder\nlearning', fontsize=9,
         ha='center', color='#1565C0', fontweight='bold')
ax4.text(32, max(total_curve) * 0.9, 'Phase 2:\nEncoder\norganizing', fontsize=9,
         ha='center', color='#E65100', fontweight='bold')
ax4.text(75, max(total_curve) * 0.9, 'Phase 3:\nEquilibrium', fontsize=9,
         ha='center', color='#2E7D32', fontweight='bold')

ax4.set_xlabel('Epoch', fontsize=12)
ax4.set_ylabel('Loss', fontsize=12)
ax4.set_title('VAE Training Dynamics: How the ELBO Evolves', fontsize=14, fontweight='bold')
ax4.legend(fontsize=11, loc='center right')
ax4.grid(True, alpha=0.3)

# ================================================================
# Panel 5 (bottom-left): Reconstruction quality vs KL for different beta
# ================================================================
ax5 = fig.add_subplot(gs[2, 0])

betas_demo = [0.1, 0.5, 1.0, 2.0, 5.0, 10.0]
# Simulated: as beta increases, recon gets worse but KL gets better
recon_at_beta = [80, 90, 105, 130, 180, 250]
kl_at_beta = [15, 8, 5, 3, 1.5, 0.5]
colors_beta = plt.cm.coolwarm(np.linspace(0, 1, len(betas_demo)))

for i, (b, r, k) in enumerate(zip(betas_demo, recon_at_beta, kl_at_beta)):
    ax5.scatter(k, r, color=colors_beta[i], s=120, zorder=5, edgecolors='black', linewidth=1)
    ax5.annotate(f'$\\beta$={b}', xy=(k, r), xytext=(k + 0.8, r + 5), fontsize=9)

ax5.set_xlabel('KL Divergence', fontsize=11)
ax5.set_ylabel('Reconstruction Loss', fontsize=11)
ax5.set_title('Recon-KL Tradeoff\n(Pareto frontier)', fontsize=12, fontweight='bold')
ax5.grid(True, alpha=0.3)

# ================================================================
# Panel 6 (bottom-center): Encoder distributions at different KLs
# ================================================================
ax6 = fig.add_subplot(gs[2, 1])

z_range = np.linspace(-5, 5, 300)

# Standard normal prior
prior = (1 / np.sqrt(2*np.pi)) * np.exp(-0.5 * z_range**2)
ax6.fill_between(z_range, prior, alpha=0.15, color='gray')
ax6.plot(z_range, prior, color='gray', linewidth=2, linestyle='--', label='Prior $p(z) = \\mathcal{N}(0,1)$')

# Encoder distributions with different KL values
configs = [
    (0.0, 1.0, '#4CAF50', 'KL $\\approx$ 0'),
    (0.5, 1.2, '#FF9800', 'KL = 0.16'),
    (2.0, 0.5, '#E91E63', 'KL = 2.12'),
]
for mu_c, sigma_c, color_c, label_c in configs:
    pdf_c = (1 / (sigma_c * np.sqrt(2*np.pi))) * np.exp(-0.5 * ((z_range - mu_c)/sigma_c)**2)
    ax6.plot(z_range, pdf_c, color=color_c, linewidth=2, label=label_c)

ax6.set_xlabel('$z$', fontsize=11)
ax6.set_ylabel('Density', fontsize=11)
ax6.set_title('Encoder Distributions\nat Different KL Values', fontsize=12, fontweight='bold')
ax6.legend(fontsize=9)
ax6.set_xlim(-5, 5)

# ================================================================
# Panel 7 (bottom-right): ELBO decomposition pie charts
# ================================================================
ax7 = fig.add_subplot(gs[2, 2])

# Show ELBO composition at epoch 5, 30, 100
epochs_show = ['Epoch 5', 'Epoch 30', 'Epoch 100']
recon_vals = [95, 70, 55]
kl_vals = [5, 30, 45]

x_pos = np.arange(len(epochs_show))
width = 0.6

ax7.bar(x_pos, recon_vals, width, color='#2196F3', label='Reconstruction %')
ax7.bar(x_pos, kl_vals, width, bottom=recon_vals, color='#FF5722', label='KL Divergence %')

for i in range(len(epochs_show)):
    ax7.text(x_pos[i], recon_vals[i]/2, f'{recon_vals[i]}%', ha='center', va='center',
             fontsize=11, fontweight='bold', color='white')
    ax7.text(x_pos[i], recon_vals[i] + kl_vals[i]/2, f'{kl_vals[i]}%', ha='center',
             va='center', fontsize=11, fontweight='bold', color='white')

ax7.set_xticks(x_pos)
ax7.set_xticklabels(epochs_show, fontsize=11)
ax7.set_ylabel('Share of Total Loss (%)', fontsize=11)
ax7.set_title('ELBO Composition\nOver Training', fontsize=12, fontweight='bold')
ax7.legend(fontsize=10, loc='lower right')
ax7.set_ylim(0, 110)

plt.suptitle('VAE Loss Dashboard: ELBO, KL Divergence & Reparameterization',
             fontsize=16, fontweight='bold', y=1.01)
plt.show()

print("This dashboard shows the complete picture of VAE training:")
print("  1. Reconstruction loss penalizes bad pixel predictions (top-left)")
print("  2. KL divergence penalizes deviations from the standard normal (top-center)")
print("  3. The reparameterization trick enables gradient flow (top-right)")
print("  4. Training balances reconstruction and regularization (middle)")
print("  5. Beta controls the tradeoff (bottom-left)")
print("  6. Different KL values produce different encoder shapes (bottom-center)")
print("  7. The relative importance of each term shifts during training (bottom-right)")

## 9. Reflection Questions

Take a moment to reflect on what we have learned. Try to answer these questions *before* looking at the hints.

**Question 1:** If we set the KL divergence weight to zero (effectively removing the regularization term), what would happen to the latent space? Would the model still be able to generate new samples?

> *Hint: Think about what happens to a regular autoencoder. Without the KL term, what prevents the encoder from mapping each image to an isolated, arbitrary point?*

**Question 2:** The reparameterization trick separates randomness ($\epsilon$) from learned parameters ($\mu$, $\sigma$). Can you think of another domain where a similar "separation of randomness" technique is used?

> *Hint: Think about dropout in neural networks, or REINFORCE vs. the Gumbel-Softmax trick in reinforcement learning.*

**Question 3:** In the KL divergence formula $D_{KL} = -\frac{1}{2}(1 + \log\sigma^2 - \mu^2 - \sigma^2)$, what happens when $\sigma \to 0$? What does this mean for the encoder's behavior?

> *Hint: Compute $\lim_{\sigma \to 0} \log(\sigma^2)$. What does an infinitely narrow distribution mean?*

### Optional Challenges

If you want to go deeper, try these:

1. **Derive the KL divergence yourself.** Start from the definition $D_{KL}(q \| p) = \mathbb{E}_q[\log q(z) - \log p(z)]$ with $q = \mathcal{N}(\mu, \sigma^2)$ and $p = \mathcal{N}(0, 1)$. Substitute the Gaussian PDFs and simplify. You should arrive at $-\frac{1}{2}(1 + \log\sigma^2 - \mu^2 - \sigma^2)$.

2. **Implement MSE reconstruction loss** instead of BCE. When would you use MSE instead of BCE? (Hint: think about continuous vs. binary data.)

3. **Explore the "posterior collapse" problem.** What happens when the KL divergence goes to zero everywhere? Is this always a good thing? (Hint: it means the decoder ignores $z$ entirely.)

## Summary

In this notebook, we covered the three pillars of VAE training:

| Component | What it does | Formula |
|-----------|-------------|---------|
| **Reconstruction Loss** | Ensures decoded images match originals | $-\sum_i [x_i \log p_i + (1-x_i)\log(1-p_i)]$ |
| **KL Divergence** | Keeps latent space organized near $\mathcal{N}(0,I)$ | $-\frac{1}{2}\sum_j (1 + \log\sigma_j^2 - \mu_j^2 - \sigma_j^2)$ |
| **Reparameterization** | Enables gradient-based training through sampling | $z = \mu + \sigma \odot \epsilon, \quad \epsilon \sim \mathcal{N}(0,I)$ |

The ELBO combines reconstruction and regularization into a single objective:

$$\mathcal{L}_{\text{VAE}} = \underbrace{-\mathbb{E}_{q_\phi(z|x)}[\log p_\theta(x|z)]}_{\text{Reconstruction}} + \underbrace{D_{KL}(q_\phi(z|x) \| p(z))}_{\text{Regularization}}$$

**In the next notebook**, we will put all of this into action by building a complete VAE in PyTorch, training it on MNIST, and visualizing the latent space.

---

*Vizuara -- Making AI Intuitive*